In [1]:
!nvidia-smiA

Tue Nov 15 06:16:52 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.27.04    Driver Version: 460.27.04    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  GeForce RTX 208...  Off  | 00000000:3B:00.0 Off |                  N/A |
| 30%   26C    P8     2W / 250W |   9359MiB / 11019MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  GeForce RTX 208...  Off  | 00000000:5E:00.0 Off |                  N/A |
| 30%   

In [2]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "3"

In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset

from sklearn.metrics import f1_score
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

import os
import pandas as pd
import numpy as np
# from tqdm.auto import tqdm
from tqdm import tqdm
import random

import warnings
warnings.filterwarnings(action='ignore') 

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
print(device)
print(torch.cuda.device_count())

cuda
1


In [4]:
CONFIG = {
    'EPOCHS': 300,
    'LEARNING_RATE':1e-3,
    'BATCH_SIZE':512,
    'SEED':101
}

In [5]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

seed_everything(CONFIG['SEED'])

In [6]:
train = pd.read_csv('./train.csv')
test = pd.read_csv('./test.csv')

# Preprocessing

In [7]:
categorical_features = ['COMPONENT_ARBITRARY', 'YEAR']
# Inference(실제 진단 환경)에 사용하는 컬럼
test_stage_features = ['COMPONENT_ARBITRARY', 'ANONYMOUS_1', 'YEAR' , 'ANONYMOUS_2', 'AG', 'CO', 'CR', 'CU', 'FE', 'H2O', 'MN', 'MO', 'NI', 'PQINDEX', 'TI', 'V', 'V40', 'ZN']

In [8]:
train = train.fillna(0)
test = test.fillna(0)

In [9]:
all_X = train.drop(['ID', 'Y_LABEL'], axis = 1)
all_y = train['Y_LABEL']

test = test.drop(['ID'], axis = 1)

train_X, val_X, train_y, val_y = train_test_split(all_X, all_y, test_size=0.2, random_state=CONFIG['SEED'], stratify=all_y)

In [10]:
# ---------------------------
# Handling Numerical
# ---------------------------
def get_values(value):
    return value.values.reshape(-1, 1)

for col in train_X.columns:
    if col not in categorical_features:
        scaler = StandardScaler()
        train_X[col] = scaler.fit_transform(get_values(train_X[col]))
        val_X[col] = scaler.transform(get_values(val_X[col]))
        if col in test.columns:
            test[col] = scaler.transform(get_values(test[col]))
            
            
# ---------------------------
# Handling Categorical
# ---------------------------
le = LabelEncoder()
for col in categorical_features:    
    train_X[col] = le.fit_transform(train_X[col])
    val_X[col] = le.transform(val_X[col])
    if col in test.columns:
        test[col] = le.transform(test[col])

# Large Model (teacher)

In [11]:
# ---------------------------
# Custom Dataset
# ---------------------------
class CustomDataset(Dataset):
    def __init__(self, data_X, data_y, distillation=False):
        super(CustomDataset, self).__init__()
        self.data_X = data_X
        self.data_y = data_y
        self.distillation = distillation
        
    def __len__(self):
        return len(self.data_X)
    
    def __getitem__(self, index):
        if self.distillation:
            #------------------------------------
            # When Learing with distillation
            #------------------------------------
            teacher_X = torch.Tensor(self.data_X.iloc[index])
            student_X = torch.Tensor(self.data_X[test_stage_features].iloc[index])
            label_y   = torch.Tensor([self.data_y.iloc[index]])
            return teacher_X.to(device), \
                   student_X.to(device), \
                   label_y.to(device)
        else:
            #------------------------------------
            # When Learing with Normal Data
            #------------------------------------
            if self.data_y is None: # for submission dataset
                test_X = torch.Tensor(self.data_X.iloc[index])
                return test_X.to(device)
            
            else:
                teacher_X = torch.Tensor(self.data_X.iloc[index])
                label_y   = torch.Tensor([self.data_y.iloc[index]])
                return teacher_X.to(device), \
                       label_y.to(device)

In [12]:
# ---------------------------
# Dataset for Teacher Model
# ---------------------------
train_dataset = CustomDataset(train_X, train_y, False)
val_dataset = CustomDataset(val_X, val_y, False)

# ---------------------------
# Dataloader for Teacher Model
# ---------------------------
train_loader = DataLoader(train_dataset, batch_size = CONFIG['BATCH_SIZE'], shuffle=True)
val_loader   = DataLoader(val_dataset, batch_size = CONFIG['BATCH_SIZE'], shuffle=False)

In [13]:
val_dataset[0]

(tensor([ 2.0000e+00, -3.9926e-01,  7.0000e+00,  3.3202e-02, -3.4073e-01,
         -1.5054e-01, -8.0990e-02,  9.7940e-01, -3.0781e-01, -4.2608e-02,
          1.5444e+00, -6.5092e-02, -8.5028e-02,  5.2004e-02, -2.0129e-01,
         -3.6416e-01, -5.8966e-01, -3.6463e-01, -5.8662e-01, -6.0484e-01,
         -6.0634e-01,  1.8625e-01, -1.0570e-01, -3.9884e-02, -3.4690e-03,
         -1.0803e-01, -1.9039e-01,  7.8440e-01, -3.0012e-01,  1.4040e+00,
         -1.8458e-01,  5.3503e-01, -1.5001e-01, -7.4769e-02, -1.2426e-01,
         -1.7229e-01, -2.8641e-02, -2.3638e-01, -3.8338e-01, -1.0043e-01,
         -7.6385e-02, -9.9691e-02, -7.2717e-02, -1.2111e-01, -1.2131e-01,
         -1.3697e-01, -1.5457e-01, -1.9158e-01, -1.1032e-01, -5.9492e-01,
         -7.4579e-01,  1.4160e+00], device='cuda:0'),
 tensor([0.], device='cuda:0'))

In [14]:
# ---------------------------
# Teacher Model
# ---------------------------
class Teacher(nn.Module):
    def __init__(self):
        super(Teacher, self).__init__()
        self.classifier = nn.Sequential(
            nn.Linear(in_features=52, out_features=256),
            nn.BatchNorm1d(256),
            nn.LeakyReLU(),
            nn.Linear(in_features=256, out_features=512),
            nn.BatchNorm1d(512),
            nn.LeakyReLU(),
            nn.Linear(in_features=512, out_features=1024),
            nn.BatchNorm1d(1024),
            nn.LeakyReLU(),
            nn.Linear(in_features=1024, out_features=1024),
            nn.BatchNorm1d(1024),
            nn.LeakyReLU(),
            nn.Linear(in_features=1024, out_features=512),
            nn.BatchNorm1d(512),
            nn.LeakyReLU(),
            nn.Linear(in_features=512, out_features=256),
            nn.BatchNorm1d(256),
            nn.LeakyReLU(),
            nn.Linear(in_features=256, out_features=1),
            nn.Sigmoid()
        )
        
    def forward(self, x):
        output = self.classifier(x)
        return output

In [15]:
#--------------------------------------------------------------------------------
# Metrics
#--------------------------------------------------------------------------------
def competition_metric(true, pred):
    return f1_score(true, pred, average="macro")


#--------------------------------------------------------------------------------
# Training Teacher
#--------------------------------------------------------------------------------
def teacher_train(model, optimizer, train_loader, val_loader, scheduler):
    model.to(device)

    best_score = 0
    best_model = None
    criterion = nn.BCELoss().to(device)

    for epoch in range(CONFIG["EPOCHS"]):
        train_loss = []
        
        # ACTIVATE TRAINING MODE
        # --- normalisation layers1 use per-batch statistics
        # --- activates Dropout layers2
        model.train()
    
        for i,(X, y) in tqdm(enumerate(train_loader)):
            
            # ZERO GRADIENT
            optimizer.zero_grad()
            
            # FORWARD
            y_pred = model(X)
            loss = criterion(y_pred, y)
            
            # BACKWARD
            loss.backward()
            
            # UPDATE
            optimizer.step()

            train_loss.append(loss.item())

        val_loss, val_score = validation_teacher(model, val_loader, criterion)
        print(f'Epoch [{epoch}], Train Loss : [{np.mean(train_loss) :.5f}] Val Loss : [{np.mean(val_loss) :.5f}] Val F1 Score : [{val_score:.5f}]')

        if scheduler is not None:
            scheduler.step(val_score)
            
        if best_score < val_score:
            best_model = model
            best_score = val_score
        
    return best_model 


#--------------------------------------------------------------------------------
# Validation Teacher
#--------------------------------------------------------------------------------
def validation_teacher(model, val_loader, criterion):
    # ACTIVATE EVALUATION MODE
    # --- normalisation layers use running statistics
    # --- de-activates Dropout layers
    model.eval()

    val_loss = []
    pred_labels = []
    true_labels = []
    threshold = 0.5
    
    with torch.no_grad():
        for i,(X, y) in tqdm(enumerate(val_loader)):
            
            model_pred = model(X)
            
            loss = criterion(model_pred,y)
            val_loss.append(loss.item())      
            
            model_pred = model_pred.squeeze(1)#.to('cpu')  
            pred_labels += model_pred.tolist()
            true_labels += y.tolist()
        
        pred_labels = np.where(np.array(pred_labels) > threshold, 1, 0)
        val_f1 = competition_metric(true_labels, pred_labels)
    return val_loss, val_f1

In [16]:
#--------------------------------------------------------------------------------
# Training Process
#--------------------------------------------------------------------------------
model = Teacher()
optimizer = torch.optim.Adam(model.parameters(), lr=CONFIG['LEARNING_RATE'])
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='max', factor=0.5, patience=10, threshold_mode='abs',min_lr=1e-8, verbose=True)

teacher_model = teacher_train(model, optimizer, train_loader, val_loader, scheduler)

23it [00:03,  5.97it/s]
6it [00:00,  6.57it/s]


Epoch [0], Train Loss : [0.30682] Val Loss : [0.21449] Val F1 Score : [0.77588]


23it [00:03,  6.05it/s]
6it [00:00,  6.06it/s]


Epoch [1], Train Loss : [0.19598] Val Loss : [0.19544] Val F1 Score : [0.77880]


23it [00:03,  6.08it/s]
6it [00:00,  6.64it/s]


...

Epoch [297], Train Loss : [0.04185] Val Loss : [0.26129] Val F1 Score : [0.76876]


23it [00:03,  6.16it/s]
6it [00:00,  6.76it/s]


Epoch [298], Train Loss : [0.04020] Val Loss : [0.26704] Val F1 Score : [0.77095]


23it [00:03,  6.15it/s]
6it [00:00,  6.78it/s]

Epoch [299], Train Loss : [0.04379] Val Loss : [0.25874] Val F1 Score : [0.77616]


# Small Model (student)

In [17]:
# ---------------------------
# Student Model
# ---------------------------
class Student(nn.Module):
    def __init__(self):
        super(Student, self).__init__()
        self.classifier = nn.Sequential(
            nn.Linear(in_features=18, out_features=128),
            nn.BatchNorm1d(128),
            nn.LeakyReLU(),
            nn.Linear(in_features=128, out_features=256),
            nn.BatchNorm1d(256),
            nn.LeakyReLU(),
            nn.Linear(in_features=256, out_features=256),
            nn.BatchNorm1d(256),
            nn.LeakyReLU(),
            nn.Linear(in_features=256, out_features=128),
            nn.BatchNorm1d(128),
            nn.LeakyReLU(),
            nn.Linear(in_features=128, out_features=1),
            nn.Sigmoid()
        )
        
    def forward(self, x):
        output = self.classifier(x)
        return output

In [18]:
# ---------------------------
# Distillation Loss
# ---------------------------

def distillation(student_logits, labels, teacher_logits, alpha):
    distillation_loss = nn.BCELoss()(student_logits, teacher_logits)
    student_loss = nn.BCELoss()(student_logits, labels.reshape(-1, 1))
    return alpha * student_loss + (1-alpha) * distillation_loss

def distill_loss(output, target, teacher_output, loss_fn=distillation, opt=optimizer):
    loss_b = loss_fn(output, target, teacher_output, alpha=0.1)

    if opt is not None:
        # BACKWARD
        loss_b.backward()
        
        # UPDATE
        opt.step()

    return loss_b.item()

In [19]:
#--------------------------------------------------------------------------------
# Training Student
#--------------------------------------------------------------------------------

def student_train(s_model, t_model, optimizer, train_loader, val_loader, scheduler, device):
    
    s_model.to(device)
    t_model.to(device)
    
    best_score = 0
    best_model = None

    for epoch in range(CONFIG["EPOCHS"]):
        train_loss = []
        s_model.train()
        t_model.eval()
        
        for X_t, X_s, y in tqdm(train_loader):
            
            # ZERO GRADIENT
            optimizer.zero_grad()
            
            # FORWARD
            y_pred_student = s_model(X_s)
            with torch.no_grad():
                y_pred_teacher = t_model(X_t)
                
            loss_b = distill_loss(y_pred_student, y, y_pred_teacher, loss_fn=distillation, opt=optimizer)
            
                
            train_loss.append(loss_b)

        val_loss, val_score = validation_student(s_model, t_model, val_loader, distill_loss, device)
        print(f'Epoch [{epoch}], Train Loss : [{np.mean(train_loss) :.5f}] Val Loss : [{np.mean(val_loss) :.5f}] Val F1 Score : [{val_score:.5f}]')
        
        if scheduler is not None:
            scheduler.step(val_score)
            
        if best_score < val_score:
            best_model = s_model
            best_score = val_score
        
    return best_model


#--------------------------------------------------------------------------------
# Validation Student
#--------------------------------------------------------------------------------

def validation_student(s_model, t_model, val_loader, criterion, device):
    s_model.eval()
    t_model.eval()

    val_loss = []
    pred_labels = []
    true_labels = []
    threshold = 0.5
    
    with torch.no_grad():
        for X_t, X_s, y in tqdm(val_loader):
            
            model_pred     = s_model(X_s)
            teacher_output = t_model(X_t)
            
            loss_b = distill_loss(model_pred, y, teacher_output, loss_fn=distillation, opt=None)
            val_loss.append(loss_b)
            
            model_pred = model_pred.squeeze(1)#.to('cpu')
            pred_labels += model_pred.tolist()
            true_labels += y.tolist()
        
        pred_labels = np.where(np.array(pred_labels) > threshold, 1, 0)
        val_f1 = competition_metric(true_labels, pred_labels)
    return val_loss, val_f1    

In [20]:
# ---------------------------
# Dataset for Student Model (Distillation True)
# ---------------------------
train_dataset = CustomDataset(train_X, train_y, True)
val_dataset = CustomDataset(val_X, val_y, True)

# ---------------------------
# Dataloader for Student Model (Distillation True)
# ---------------------------
train_loader = DataLoader(train_dataset, batch_size = CONFIG['BATCH_SIZE'], shuffle=True)
val_loader = DataLoader(val_dataset, batch_size = CONFIG['BATCH_SIZE'], shuffle=False)

In [21]:
#--------------------------------------------------------------------------------
# Training Process
#--------------------------------------------------------------------------------

student_model = Student()
# student_model.eval()
optimizer = torch.optim.Adam(student_model.parameters(), lr=CONFIG['LEARNING_RATE'])
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='max', factor=0.5, patience=10, threshold_mode='abs',min_lr=1e-8, verbose=True)

best_student_model = student_train(student_model, teacher_model, optimizer, train_loader, val_loader, scheduler, device)

100%|██████████| 6/6 [00:03<00:00,  1.75it/s]


Epoch [0], Train Loss : [0.42322] Val Loss : [0.40977] Val F1 Score : [0.48892]


100%|██████████| 6/6 [00:03<00:00,  1.76it/s]


Epoch [1], Train Loss : [0.29778] Val Loss : [0.30147] Val F1 Score : [0.47767]


100%|██████████| 6/6 [00:03<00:00,  1.77it/s]


...

Epoch [297], Train Loss : [0.27799] Val Loss : [0.30333] Val F1 Score : [0.49565]


100%|██████████| 6/6 [00:03<00:00,  1.80it/s]


Epoch [298], Train Loss : [0.25408] Val Loss : [0.30568] Val F1 Score : [0.49531]


100%|██████████| 6/6 [00:03<00:00,  1.80it/s]

Epoch [299], Train Loss : [0.26206] Val Loss : [0.30393] Val F1 Score : [0.49910]


In [ ]:
def choose_threshold(model, val_loader, device):
    model.to(device)
    model.eval()
    
    thresholds = [0.1, 0.2, 0.25, 0.3, 0.35, 0.4, 0.45, 0.5]
    pred_labels = []
    true_labels = []
    
    best_score = 0
    best_thr = None
    with torch.no_grad():
        for _, x_s, y in tqdm(iter(val_loader)):
            
            model_pred = model(x_s)
            
            model_pred = model_pred.squeeze(1).to('cpu')
            pred_labels += model_pred.tolist()
            true_labels += y.tolist()
        
        for threshold in thresholds:
            pred_labels_thr = np.where(np.array(pred_labels) > threshold, 1, 0)
            score_thr = competition_metric(true_labels, pred_labels_thr)
            if best_score < score_thr:
                best_score = score_thr
                best_thr = threshold
    return best_thr, best_score

In [ ]:
best_threshold, best_score = choose_threshold(best_student_model, val_loader, device)
print(f'Best Threshold : [{best_threshold}], Score : [{best_score:.5f}]')

In [ ]:
test_datasets = CustomDataset(test, None, False)
test_loaders = DataLoader(test_datasets, batch_size = CFG['BATCH_SIZE'], shuffle=False)

In [ ]:
def inference(model, test_loader, threshold, device):
    model.to(device)
    model.eval()
    
    test_predict = []
    with torch.no_grad():
        for x in tqdm(test_loader):
            x = x.float().to(device)
            model_pred = model(x)

            model_pred = model_pred.squeeze(1).to('cpu')
            test_predict += model_pred
        
    test_predict = np.where(np.array(test_predict) > threshold, 1, 0)
    print('Done.')
    return test_predict

In [ ]:
preds = inference(best_student_model, test_loaders, best_threshold, device)

In [ ]:
submit = pd.read_csv('./sample_submission.csv')
submit['Y_LABEL'] = preds
submit.head()

In [ ]:
submit.to_csv('./submit.csv', index=False)